In [3]:
!pip install ultralytics

# labels are: class x1 y1 x2 y2 x3 y3 x4 y4   (normalized)


     ---------------------------------------- 1.1/1.1 MB 6.8 MB/s eta 0:00:00
     ------------------------------------- 772.7/772.7 KB 24.6 MB/s eta 0:00:00
     --------------------------------------- 40.1/40.1 MB 17.7 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\jayjo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import sys, platform, torch
print('Python:', sys.version.split()[0])
print('CUDA available:', torch.cuda.is_available())
print('CUDA device count:', torch.cuda.device_count())
if torch.cuda.is_available():
    print('CUDA device:', torch.cuda.get_device_name(0))
print('Platform:', platform.platform())

c:\Users\jayjo\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


Python: 3.9.13
CUDA available: False
CUDA device count: 0
Platform: Windows-10-10.0.19045-SP0


In [5]:
%pip -q install ultralytics roboflow==1.* opencv-python matplotlib
import ultralytics, cv2, matplotlib
from ultralytics import YOLO
print('ultralytics', ultralytics.__version__)

You should consider upgrading via the 'c:\Users\jayjo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.
Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\jayjo\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
ultralytics 8.3.216


In [15]:
import os, glob, zipfile, yaml
from pathlib import Path

# 1) Your ZIP file (use raw string or double backslashes on Windows)
ZIP_PATH = r"C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\Package and label detector.v1i.yolov8.zip"

# 2) Extract to a SHORT path (helps avoid Windows MAX_PATH issues)
EXTRACT_DIR = Path(r"C:\ds_yolo")   # change if you want
EXTRACT_DIR.mkdir(parents=True, exist_ok=True)

# (Re)extract
with zipfile.ZipFile(ZIP_PATH, "r") as z:
    z.extractall(EXTRACT_DIR)

print("Unzipped to:", EXTRACT_DIR)

# 3) Find actual train/valid/test image dirs anywhere under EXTRACT_DIR
def find_unique_dir(pattern):
    hits = glob.glob(str(EXTRACT_DIR / pattern), recursive=True)
    # Keep only folders that actually exist and contain images
    hits = [h for h in hits if Path(h).is_dir() and any(Path(h).glob("*.jpg")) or any(Path(h).glob("*.png"))]
    hits = sorted(set(map(os.path.normpath, hits)))
    return hits

train_hits = find_unique_dir("**/train/images")
val_hits   = find_unique_dir("**/valid/images")
test_hits  = find_unique_dir("**/test/images")

print("Found train/images:", train_hits)
print("Found valid/images:", val_hits)
print("Found test/images: ", test_hits)

def pick_one(name, hits):
    if not hits:
        return None
    if len(hits) > 1:
        print(f"Note: multiple {name} candidates found; picking the first.")
    return str(Path(hits[0]).resolve())

train_dir = pick_one("train", train_hits)
val_dir   = pick_one("valid", val_hits)
test_dir  = pick_one("test",  test_hits)

if not train_dir or not val_dir:
    raise RuntimeError("Could not locate train/valid image folders after unzip. Check the ZIP structure.")

# 4) Try to locate a data.yaml (optional; we’ll create our own if needed)
yaml_candidates = glob.glob(str(EXTRACT_DIR / "**" / "data.yaml"), recursive=True)
names = None
if yaml_candidates:
    try:
        cfg0 = yaml.safe_load(Path(yaml_candidates[0]).read_text(encoding="utf-8"))
        names = cfg0.get("names", None)
    except Exception:
        pass

# If class names aren’t found, fall back—adjust if your classes differ
if names is None:
    names = ["Boxes", "label", "package"]  # update if needed

# 5) Write a patched absolute-path config
patched = {
    "path": str(EXTRACT_DIR.resolve()),  # optional
    "train": train_dir,
    "val":   val_dir,
    "test":  test_dir,
    "names": names,
}

PATCHED_YAML = EXTRACT_DIR / "data_local.yaml"
PATCHED_YAML.write_text(yaml.safe_dump(patched, sort_keys=False, allow_unicode=True), encoding="utf-8")
print("Wrote:", PATCHED_YAML)

# 6) Print CLI & Python commands that will now work
print("\nTrain with CLI:")
print(f'yolo detect train data="{PATCHED_YAML}" model=yolov8n.pt imgsz=640 epochs=100 batch=16 name=labels-v8')

print("\nOr in Python:")
print(f'from ultralytics import YOLO\nm = YOLO("yolov8n.pt")\nm.train(data=r"{PATCHED_YAML}", epochs=100, imgsz=640, batch=16, name="labels-v8")')


Unzipped to: C:\ds_yolo
Found train/images: ['C:\\ds_yolo\\train\\images']
Found valid/images: ['C:\\ds_yolo\\valid\\images']
Found test/images:  ['C:\\ds_yolo\\test\\images']
Wrote: C:\ds_yolo\data_local.yaml

Train with CLI:
yolo detect train data="C:\ds_yolo\data_local.yaml" model=yolov8n.pt imgsz=640 epochs=100 batch=16 name=labels-v8

Or in Python:
from ultralytics import YOLO
m = YOLO("yolov8n.pt")
m.train(data=r"C:\ds_yolo\data_local.yaml", epochs=100, imgsz=640, batch=16, name="labels-v8")


In [20]:
import yaml
from pathlib import Path

ORIG_YAML = r"C:\\Users\\jayjo\\Downloads\\QualitiyControl\\QualityControl-PBA\\backend\\data_local.yaml"
cfg = yaml.safe_load(Path(ORIG_YAML).read_text(encoding="utf-8"))

print("YAML keys:", list(cfg.keys()))
for k in ("train","val","valid","test"):
    if k in cfg:
        print(f"{k} =", cfg[k])
print("names =", cfg.get("names"))


YAML keys: ['train', 'val', 'test', 'nc', 'names', 'roboflow']
train = /mnt/data/label_dataset/train/images
val = /mnt/data/label_dataset/valid/images
test = /mnt/data/label_dataset/test/images
names = ['Boxes', 'label', 'package']


In [24]:
import os, zipfile, glob, random, shutil, yaml
from pathlib import Path

# === EDIT THESE ===
ZIP_PATH      = r"C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\Package and label detector.v1i.yolov8.zip"
EXTRACT_DIR   = Path(r"C:\ds_yolo")     # short, simple path helps on Windows
MAKE_LABEL_ONLY = True                  # set False if you want to keep 3 classes
LABEL_CLASS_ID  = 1                     # 'label' class id in original set (Boxes=0,label=1,package=2)
VAL_FRACTION    = 0.15                  # if val/valid missing, sample this fraction from train
# ===================

EXTRACT_DIR.mkdir(parents=True, exist_ok=True)

# --- 1) Unzip fresh
with zipfile.ZipFile(ZIP_PATH, "r") as z:
    z.extractall(EXTRACT_DIR)
print("Unzipped to:", EXTRACT_DIR)

# --- helpers
IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp",
            ".JPG",".JPEG",".PNG",".BMP",".TIF",".TIFF",".WEBP")

def has_images(p: Path) -> bool:
    return p.is_dir() and any(p.glob("*.jpg")) or any(p.glob("*.png"))

def find_dir(pattern: str):
    hits = [Path(h) for h in glob.glob(str(EXTRACT_DIR / pattern), recursive=True)]
    hits = [h for h in hits if h.is_dir()]
    # keep only those with images
    hits = [h for h in hits if any(h.glob("*.jpg")) or any(h.glob("*.png"))]
    hits = sorted(set(h.resolve() for h in hits))
    return hits[0] if hits else None

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

# --- 2) Locate splits
train_images = find_dir("**/train/images")
val_images   = find_dir("**/valid/images") or find_dir("**/val/images")
test_images  = find_dir("**/test/images")

print("Found train/images:", train_images)
print("Found val/images:  ", val_images)
print("Found test/images: ", test_images)

if not train_images:
    raise RuntimeError("Could not find train/images anywhere under EXTRACT_DIR. Check ZIP or change EXTRACT_DIR.")

# --- 3) Ensure a labels folder exists next to each images folder
def labels_dir_for(images_dir: Path) -> Path:
    # expected at ../labels
    d = images_dir.parent / "labels"
    if d.exists():
        return d
    # try sibling search
    sib = list(images_dir.parent.glob("labels"))
    return sib[0] if sib else d  # may not exist yet

train_labels = labels_dir_for(train_images)
if not train_labels.exists():
    raise RuntimeError(f"No labels folder found for train near {train_images}")

# --- 4) If val/valid missing, create val by sampling train
if not val_images:
    base = train_images.parent.parent       # .../<split_root>
    val_images = base / "val" / "images"
    val_labels = base / "val" / "labels"
    ensure_dir(val_images); ensure_dir(val_labels)

    # sample from train
    imgs = [p for p in train_images.iterdir() if p.suffix in IMG_EXTS]
    if not imgs:
        raise RuntimeError("Train has no images to sample from.")
    random.seed(42)
    k = max(1, int(VAL_FRACTION * len(imgs)))
    sample = set(random.sample(imgs, k))

    moved = 0
    for img in sample:
        stem = img.stem
        lbl  = train_labels / f"{stem}.txt"
        if not lbl.exists():
            continue
        shutil.copy2(img,  val_images / img.name)
        shutil.copy2(lbl,  val_labels / f"{stem}.txt")
        moved += 1
    print(f"Created val split at: {val_images}  (moved ~{moved} images)")

# Confirm val labels
val_labels = labels_dir_for(val_images)
print("Train labels:", train_labels)
print("Val labels:  ", val_labels)

# --- 5) Class names: try to read from any data.yaml found; else fallback
names = None
yaml_candidates = glob.glob(str(EXTRACT_DIR / "**" / "data.yaml"), recursive=True)
if yaml_candidates:
    try:
        cfg0 = yaml.safe_load(Path(yaml_candidates[0]).read_text(encoding="utf-8"))
        if isinstance(cfg0.get("names"), list) and cfg0["names"]:
            names = cfg0["names"]
    except Exception:
        pass
if not names:
    names = ["Boxes","label","package"]
print("Class names:", names)

# --- 6) Write patched YAML with absolute paths
patched = {
    "train": str(train_images.resolve()),
    "val":   str(val_images.resolve()),
    "names": names,
}
if test_images:
    patched["test"] = str(test_images.resolve())

PATCHED_YAML = EXTRACT_DIR / "data_local_patched.yaml"
PATCHED_YAML.write_text(yaml.safe_dump(patched, sort_keys=False, allow_unicode=True), encoding="utf-8")
print("Wrote patched YAML:", PATCHED_YAML)

# === OPTIONAL: 7) Build a single-class (label-only) dataset and YAML ===
if MAKE_LABEL_ONLY:
    OUT_ROOT = EXTRACT_DIR / "ds_yolo_label_only"
    for split in ("train","val","test"):
        d = patched.get(split)
        if not d: 
            continue
        (OUT_ROOT / split / "images").mkdir(parents=True, exist_ok=True)
        (OUT_ROOT / split / "labels").mkdir(parents=True, exist_ok=True)

    def match_img(images_dir: Path, stem: str):
        for ext in IMG_EXTS:
            p = images_dir / f"{stem}{ext}"
            if p.exists():
                return p
        return None

    kept_imgs, kept_boxes = {}, {}
    for split in ("train","val","test"):
        images_path = patched.get(split)
        if not images_path:
            continue
        images_dir = Path(images_path)
        labels_dir = labels_dir_for(images_dir)
        if not labels_dir.exists():
            print(f"[WARN] No labels near: {images_dir}")
            continue

        kept_imgs[split] = 0
        kept_boxes[split] = 0

        for lf in sorted(labels_dir.glob("*.txt")):
            try:
                lines = lf.read_text(encoding="utf-8").strip().splitlines()
            except Exception:
                continue
            keep = []
            for line in lines:
                parts = line.strip().split()
                if not parts:
                    continue
                # First token is class id for detect/segment/obb formats
                try:
                    cls_id = int(parts[0])
                except ValueError:
                    continue
                if cls_id == LABEL_CLASS_ID:
                    parts[0] = "0"  # remap to single-class 'label'
                    keep.append(" ".join(parts))
            if not keep:
                continue

            stem = lf.stem
            img_path = match_img(images_dir, stem) or match_img(images_dir.parent, stem)
            if not img_path:
                print(f"[MISS IMG] For label: {lf}")
                continue

            out_img = OUT_ROOT / split / "images" / img_path.name
            out_lbl = OUT_ROOT / split / "labels" / f"{stem}.txt"
            if not out_img.exists():
                shutil.copy2(img_path, out_img)
                kept_imgs[split] += 1
            out_lbl.write_text("\n".join(keep) + "\n", encoding="utf-8")
            kept_boxes[split] += len(keep)

    print("Kept images (label-only):", kept_imgs)
    print("Kept boxes  (label-only):", kept_boxes)

    label_only_yaml = {
        "train": str((OUT_ROOT / "train" / "images").resolve()),
        "names": ["label"],
    }
    if (OUT_ROOT / "val" / "images").exists() and any((OUT_ROOT / "val" / "images").iterdir()):
        label_only_yaml["val"] = str((OUT_ROOT / "val" / "images").resolve())
    if (OUT_ROOT / "test" / "images").exists() and any((OUT_ROOT / "test" / "images").iterdir()):
        label_only_yaml["test"] = str((OUT_ROOT / "test" / "images").resolve())

    OUT_YAML = OUT_ROOT / "data_label_only.yaml"
    OUT_YAML.write_text(yaml.safe_dump(label_only_yaml, sort_keys=False, allow_unicode=True), encoding="utf-8")
    print("Wrote label-only YAML:", OUT_YAML)


Unzipped to: C:\ds_yolo
Found train/images: C:\ds_yolo\train\images
Found val/images:   C:\ds_yolo\valid\images
Found test/images:  C:\ds_yolo\test\images
Train labels: C:\ds_yolo\train\labels
Val labels:   C:\ds_yolo\valid\labels
Class names: ['Boxes', 'label', 'package']
Wrote patched YAML: C:\ds_yolo\data_local_patched.yaml
Kept images (label-only): {'train': 264, 'val': 77, 'test': 44}
Kept boxes  (label-only): {'train': 437, 'val': 129, 'test': 77}
Wrote label-only YAML: C:\ds_yolo\ds_yolo_label_only\data_label_only.yaml


In [28]:
import os, random, shutil, yaml
from pathlib import Path

# === EDIT THIS to your actual label-only folder ===
ROOT = Path(r"C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\ds_yolo\ds_yolo_label_only")

IMG_EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp",".JPG",".JPEG",".PNG",".BMP",".TIF",".TIFF",".WEBP"}

def has_images(p: Path) -> bool:
    return p.is_dir() and any(x.suffix in IMG_EXTS for x in p.iterdir())

train_images = ROOT / "train" / "images"
train_labels = ROOT / "train" / "labels"
val_images   = ROOT / "val"   / "images"
val_labels   = ROOT / "val"   / "labels"
test_images  = ROOT / "test"  / "images"
test_labels  = ROOT / "test"  / "labels"

# 1) Sanity: must have train/images + train/labels with files
if not has_images(train_images):
    raise SystemExit(f"Missing or empty {train_images}")
if not train_labels.exists():
    raise SystemExit(f"Missing labels folder {train_labels}")

# 2) Ensure val exists; if not, make one by sampling 15% from train
if not has_images(val_images):
    val_images.mkdir(parents=True, exist_ok=True)
    val_labels.mkdir(parents=True, exist_ok=True)

    imgs = [p for p in train_images.iterdir() if p.suffix in IMG_EXTS]
    if not imgs:
        raise SystemExit("No images in train.")
    random.seed(42)
    k = max(1, int(0.15 * len(imgs)))
    sample = set(random.sample(imgs, k))

    moved = 0
    for img in sample:
        stem = img.stem
        lbl = train_labels / f"{stem}.txt"
        if not lbl.exists():
            continue
        # copy (not move) to keep train intact
        shutil.copy2(img,  val_images / img.name)
        shutil.copy2(lbl,  val_labels / f"{stem}.txt")
        moved += 1
    print(f"Created/filled val: {val_images}  ({moved} images)")

# 3) If test exists but empty, drop it from YAML later
has_test = has_images(test_images)

# 4) Write a clean YAML with ABSOLUTE paths that actually exist
cfg = {
    "train": str(train_images.resolve()),
    "val":   str(val_images.resolve()),
    "names": ["label"],   # single-class label-only set
}
if has_test:
    cfg["test"] = str(test_images.resolve())

OUT_YAML = ROOT / "data_label_only.yaml"
OUT_YAML.write_text(yaml.safe_dump(cfg, sort_keys=False, allow_unicode=True), encoding="utf-8")
print("Wrote:", OUT_YAML)
print("train ->", cfg["train"])
print("val   ->", cfg["val"])
print("test  ->", cfg.get("test"))


Wrote: C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\ds_yolo\ds_yolo_label_only\data_label_only.yaml
train -> C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\ds_yolo\ds_yolo_label_only\train\images
val   -> C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\ds_yolo\ds_yolo_label_only\val\images
test  -> C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\ds_yolo\ds_yolo_label_only\test\images


In [30]:
from pathlib import Path

LABELS = Path(r"C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\ds_yolo\ds_yolo_label_only\train\labels")
for lf in list(LABELS.glob("*.txt"))[:5]:
    line = lf.read_text(encoding="utf-8").strip().splitlines()[0]
    n = len(line.split())
    print(lf.name, "-> tokens:", n, "| format:", "DETECTION (5 tokens)" if n==5 else "SEGMENTATION (>5 tokens)")


00b47f821f2b5ca1854c86ef_png_jpg.rf.96b43875b1d34891314f3b9425a6446d.txt -> tokens: 13 | format: SEGMENTATION (>5 tokens)
00d0c7964e5a3ec327fb1826_png_jpg.rf.5f317317dbbbfa14d491d3fd76d86c66.txt -> tokens: 5 | format: DETECTION (5 tokens)
00d705214d388dba5aba6fba_png_jpg.rf.0ed0f4a500c60807a14a47fa7b4b3c38.txt -> tokens: 5 | format: DETECTION (5 tokens)
00e6a62b5af603c4b92e344c_png_jpg.rf.06fff19dcd687eacd7ef28dc16880924.txt -> tokens: 5 | format: DETECTION (5 tokens)
01b9f2499c343cf165bfa91f_png_jpg.rf.75747f2642bb0b4994c031cb1fa7cdd9.txt -> tokens: 11 | format: SEGMENTATION (>5 tokens)


In [32]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
model.train(
    data=r"C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\ds_yolo\ds_yolo_label_only\data_label_only.yaml",
    imgsz=640, epochs=100, batch=16, name="labels_only_v8"
)



Ultralytics 8.3.216  Python-3.9.13 torch-2.6.0+cpu CPU (Intel Core i7-10750H 2.60GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\ds_yolo\ds_yolo_label_only\data_label_only.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=labels_only_v82, nbs=64, nms=False, opset=None, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001BD1879FD30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [33]:
pip install ultralytics easyocr opencv-python pillow

     ---------------------------------------- 2.9/2.9 MB 4.5 MB/s eta 0:00:00
     -------------------------------------- 110.8/110.8 KB 3.2 MB/s eta 0:00:00
     -------------------------------------- 160.6/160.6 KB 4.7 MB/s eta 0:00:00
     -------------------------------------- 310.0/310.0 KB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\jayjo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [38]:
import cv2, numpy as np

def tighten_crop(bgr, margin=6):
    # Find the largest texty region (edges + morphology)
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    edges = cv2.Canny(blur, 50, 150)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,3))
    dil = cv2.dilate(edges, kernel, iterations=1)

    cnts, _ = cv2.findContours(dil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts: 
        return bgr
    # pick the largest contour inside the crop
    c = max(cnts, key=cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
    H, W = bgr.shape[:2]
    x1 = max(0, x - margin); y1 = max(0, y - margin)
    x2 = min(W-1, x + w + margin); y2 = min(H-1, y + h + margin)
    return bgr[y1:y2, x1:x2]


In [39]:
def preprocess_variants(bgr):
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    # Upscale
    up = cv2.resize(gray, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)

    # A) Adaptive mean
    ada_mean = cv2.adaptiveThreshold(
        cv2.fastNlMeansDenoising(up, h=7, templateWindowSize=7, searchWindowSize=21),
        255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)

    # B) Otsu
    _, otsu = cv2.threshold(up, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # C) CLAHE + Otsu
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(up)
    _, clahe_otsu = cv2.threshold(clahe, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # D) Unsharp + Otsu (sharpen)
    ga = cv2.GaussianBlur(up, (0,0), sigmaX=1.0)
    unsharp = cv2.addWeighted(up, 1.5, ga, -0.5, 0)
    _, unsharp_otsu = cv2.threshold(unsharp, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    return [ada_mean, otsu, clahe_otsu, unsharp_otsu]

def deskew(binary_img):
    # estimate skew and rotate to horizontal
    coords = np.column_stack(np.where(binary_img < 255))
    if coords.size == 0:
        return binary_img
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45: angle = -(90 + angle)
    else: angle = -angle
    (h, w) = binary_img.shape
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
    return cv2.warpAffine(binary_img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)


In [40]:
def run_easyocr(reader, img_list):
    best = {"text":"", "prob":0.0, "lines":[]}
    for im in img_list:
        rot = deskew(im)
        out = reader.readtext(rot, detail=1, paragraph=True)
        lines = []
        for item in out:
            if isinstance(item, (list, tuple)):
                if len(item) == 3: _, text, prob = item
                elif len(item) == 2: _, text = item; prob = 1.0
                else: continue
            else:
                text, prob = str(item), 1.0
            text = (text or "").strip()
            if text:
                lines.append((text, float(prob)))
        # score: combine prob and length
        if lines:
            joined = " ".join(t for t,_ in lines)
            avgp = sum(p for _,p in lines) / len(lines)
            score = avgp * (1 + np.log1p(len(joined)))
            if score > best["prob"]:
                best = {"text": joined, "prob": score, "lines":[{"text":t,"prob":p} for t,p in lines]}
    return best


In [41]:
# pip install ultralytics easyocr opencv-python pillow
from ultralytics import YOLO
import easyocr, cv2, numpy as np
from pathlib import Path

MODEL_PATH = r"C:\\Users\\jayjo\\Downloads\\QualitiyControl\\QualityControl-PBA\\backend\\runs\\detect\\labels_only_v82\\weights\\best.pt"   # update if your run name differs
SOURCE     = r"C:\\Users\\jayjo\\Downloads\\QualitiyControl\\QualityControl-PBA\\backend\\ds_yolo\\ds_yolo_label_only\\val\\images\\IMG20211013160130_jpg.rf.4c876182a75976727cd3fe9e7dd127b1.jpg"              # image file or folder

# 1) Load detector + OCR
detector = YOLO(MODEL_PATH)
ocr = easyocr.Reader(['en'])  # add languages as needed, e.g. ['en','es']

# 2) Inference (keep only confident boxes)
results = detector.predict(source=SOURCE, conf=0.25, save=False)  # save=True if you want boxed images

def preprocess_for_ocr(bgr, pad=8):
    # pad a bit, convert to gray, upscale, and threshold; tweak if needed
    h, w = bgr.shape[:2]
    bgr = cv2.copyMakeBorder(bgr, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    up   = cv2.resize(gray, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
    den  = cv2.fastNlMeansDenoising(up, h=7, templateWindowSize=7, searchWindowSize=21)
    thr  = cv2.adaptiveThreshold(den, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                 cv2.THRESH_BINARY, 21, 10)
    return thr

all_outputs = []
for r in results:
    img_path = r.path
    img_bgr  = cv2.imread(str(img_path))
    if img_bgr is None:
        continue

    detections = []
    if r.boxes is not None and len(r.boxes) > 0:
        xyxy   = r.boxes.xyxy.cpu().numpy().astype(int)
        conf   = r.boxes.conf.cpu().numpy()
        for (x1,y1,x2,y2), p in zip(xyxy, conf):
            # 3) Crop with safe bounds
            h, w = img_bgr.shape[:2]
            x1 = max(0, x1); y1 = max(0, y1); x2 = min(w-1, x2); y2 = min(h-1, y2)
            crop = img_bgr[y1:y2, x1:x2].copy()
            if crop.size == 0:
                continue

            # 4) Preprocess and OCR
            proc = preprocess_for_ocr(crop)
            # --- replace just this OCR parsing block ---

            ocr_result = ocr.readtext(proc, detail=1, paragraph=True)

            texts = []
            for item in ocr_result:
                # Expected usually: (bbox, text, prob)
                # Sometimes: (bbox, text)
                # If detail=0: item is just 'text'
                if isinstance(item, (list, tuple)):
                    if len(item) == 3:
                        bbox, text, prob = item
                    elif len(item) == 2:
                        bbox, text = item
                        prob = 1.0    # assume high confidence if missing
                    else:
                        continue
                else:
                    # detail=0 case: item is a string
                    bbox, text, prob = None, str(item), 1.0

                text = (text or "").strip()
                if text and prob >= 0.35:  # tweak threshold as needed
                    texts.append((text, float(prob)))
                            # before OCR:
                crop = tighten_crop(crop, margin=8)
                variants = preprocess_variants(crop)
                best = run_easyocr(ocr, variants)

                detections.append({
                    "box": [int(x1), int(y1), int(x2), int(y2)],
                    "det_conf": float(p),
                    "ocr_text": best["text"],
                    "ocr_lines": best["lines"],
                })


            

    all_outputs.append({"image": str(img_path), "detections": detections})

# Pretty print
for out in all_outputs:
    print("\nImage:", out["image"])
    for d in out["detections"]:
        print(f"  Box={d['box']} det_conf={d['det_conf']:.2f}  OCR='{d['ocr_text']}'")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



image 1/1 C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\ds_yolo\ds_yolo_label_only\val\images\IMG20211013160130_jpg.rf.4c876182a75976727cd3fe9e7dd127b1.jpg: 640x640 1 label, 70.9ms
Speed: 3.2ms preprocess, 70.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

Image: C:\Users\jayjo\Downloads\QualitiyControl\QualityControl-PBA\backend\ds_yolo\ds_yolo_label_only\val\images\IMG20211013160130_jpg.rf.4c876182a75976727cd3fe9e7dd127b1.jpg
  Box=[0, 0, 639, 577] det_conf=0.78  OCR='"rini2 Rdl ) 3840) VIC` Snler :'
  Box=[0, 0, 639, 577] det_conf=0.78  OCR='"rini2 Rdl ) 38+0 Senuer: VIC`'
  Box=[0, 0, 639, 577] det_conf=0.78  OCR='~rinie Rdl ) 3840) Senuer: VIC`'
  Box=[0, 0, 639, 577] det_conf=0.78  OCR='5 ;'


In [43]:
!pip install "paddleocr>=2.6.1" paddlepaddle
# For CPU on Windows, if pip complains, see PaddlePaddle install page for a matching wheel.


     ---------------------------------------- 81.0/81.0 KB 4.7 MB/s eta 0:00:00
     ------------------------------------- 101.5/101.5 MB 19.8 MB/s eta 0:00:00
     ---------------------------------------- 1.8/1.8 MB 19.5 MB/s eta 0:00:00
     ---------------------------------------- 320.2/320.2 KB ? eta 0:00:00
  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl (162 kB)
     ---------------------------------------- 119.7/119.7 KB ? eta 0:00:00
     ---------------------------------------- 43.8/43.8 KB ? eta 0:00:00
     ---------------------------------------- 5.9/5.9 MB 19.0 MB/s eta 0:00:00
     ------------------------------------- 199.4/199.4 KB 11.8 MB/s eta 0:00:00
     ---------------------------------------- 63.0/63.0 KB 3.3 MB/s eta 0:00:00
     --------------------------------------- 45.5/45.5 MB 22.6 MB/s eta 0:00:00
     ---------------------------------------- 352.6/352.6 KB ? eta 0:00:00
     -------------------------------------- 118.7/118.7 KB 7.2 MB/s eta 0:00:00
   

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\jayjo\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages\\~aml\\_yaml.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'C:\Users\jayjo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [46]:
!python -m pip install "paddlepaddle>=2.5,<3.0" -f https://www.paddlepaddle.org.cn/whl/windows/mkl/avx/stable.html


Looking in links: https://www.paddlepaddle.org.cn/whl/windows/mkl/avx/stable.html
     --------------------------------------- 81.0/81.0 MB 17.7 MB/s eta 0:00:00
     ------------------------------------- 904.2/904.2 KB 55.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
You should consider upgrading via the 'c:\Users\jayjo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [47]:
!python -m pip install "paddleocr>=2.6.1"


  Using cached paddleocr-3.3.0-py3-none-any.whl (81 kB)
  Using cached paddlex-3.3.3-py3-none-any.whl (1.8 MB)
  Using cached prettytable-3.16.0-py3-none-any.whl (33 kB)
  Using cached aistudio_sdk-0.3.8-py3-none-any.whl (62 kB)
  Using cached chardet-5.2.0-py3-none-any.whl (199 kB)
  Using cached modelscope-1.31.0-py3-none-any.whl (5.9 MB)
  Using cached ruamel.yaml-0.18.15-py3-none-any.whl (119 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)
  Using cached imagesize-1.4.1-py2.py3-none-any.whl (8.8 kB)
  Using cached bce_python_sdk-0.9.46-py3-none-any.whl (352 kB)
  Using cached future-1.0.0-py3-none-any.whl (491 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\jayjo\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Users\jayjo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [54]:
from paddleocr import PaddleOCR
import cv2

# Detect + read English, with angle classification
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # first call downloads models
#img_path = r"C:\\Users\\jayjo\\Downloads\\QualitiyControl\\QualityControl-PBA\\backend\\ds_yolo\\ds_yolo_label_only\\val\\images\\IMG20211013160130_jpg.rf.4c876182a75976727cd3fe9e7dd127b1.jpg"
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_angle_cls=True, lang='en')  # downloads models on first run
img_path = r"C:\Users\jayjo\Downloads\processed-6A24843A-5CBA-4465-8854-81195F25D0D4.jpeg"

result = ocr.ocr(img_path, cls=True)  # <-- use img_path here
words = []
for poly, (text, prob) in result[0]:
    words.append({"poly": poly, "text": text, "prob": float(prob)})

print(words)



[2025/10/16 19:33:04] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\jayjo/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\jayjo/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, re

In [55]:
from paddleocr import PaddleOCR
import cv2

ocr = PaddleOCR(use_angle_cls=True, lang='en')
img_path = r"C:\Users\jayjo\Downloads\processed-6A24843A-5CBA-4465-8854-81195F25D0D4.jpeg"

img = cv2.imread(img_path)                 # read actual image
assert img is not None, f"Couldn't read {img_path}"
result = ocr.ocr(img, cls=True)            # now pass the image array

words = []
for poly, (text, prob) in result[0]:
    words.append({"poly": poly, "text": text, "prob": float(prob)})

print(words)


[2025/10/16 19:34:27] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\jayjo/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\jayjo/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, re

In [50]:
import sys
print(sys.executable)  # this is the Python your kernel uses

# Install into THIS exact interpreter:
!{sys.executable} -m pip install --upgrade pip setuptools wheel
!{sys.executable} -m pip install "paddlepaddle==2.5.2" -f https://www.paddlepaddle.org.cn/whl/windows/mkl/avx/stable.html
!{sys.executable} -m pip install "paddleocr==2.6.1.3"

# Restart the kernel after installs, then:
from paddleocr import PaddleOCR
PaddleOCR(use_angle_cls=True, lang='en')
print("OK")


c:\Users\jayjo\AppData\Local\Programs\Python\Python39\python.exe
     ---------------------------------------- 1.8/1.8 MB 18.5 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 25.3 MB/s eta 0:00:00
  Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
  Attempting uninstall: wheel
    Found existing installation: wheel 0.43.0
    Uninstalling wheel-0.43.0:
      Successfully uninstalled wheel-0.43.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.8.0
    Uninstalling setuptools-75.8.0:
      Successfully uninstalled setuptools-75.8.0
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


Looking in links: https://www.paddlepaddle.org.cn/whl/windows/mkl/avx/stable.html
     ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
     ---------------------------------------- 0.3/72.0 MB ? eta -:--:--
     ---------------------------------------- 0.3/72.0 MB ? eta -:--:--
     --------------------------------------- 0.5/72.0 MB 558.9 kB/s eta 0:02:08
      --------------------------------------- 1.0/72.0 MB 1.1 MB/s eta 0:01:05
      --------------------------------------- 1.0/72.0 MB 1.1 MB/s eta 0:01:05
     - -------------------------------------- 1.8/72.0 MB 1.4 MB/s eta 0:00:49
     - ------------------

  You can safely remove it manually.


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to C:\Users\jayjo/.paddleocr/whl\det\en\en_PP-OCRv3_det_infer\en_PP-OCRv3_det_infer.tar


...

100%|██████████| 9.96M/9.96M [00:17<00:00, 559kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to C:\Users\jayjo/.paddleocr/whl\cls\ch_ppocr_mobile_v2.0_cls_infer\ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:08<00:00, 267kiB/s] 

[2025/10/16 19:28:21] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\jayjo/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\jayjo/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, re

OK
